In [132]:
import psycopg2

import psycopg2
import logging

DB_NAME = 'blockchain'
DB_USER = 'postgres'
DB_PASSWORD = 'auth2020'
DB_HOST = 'localhost'
DB_PORT = '5432'


try:
    # Create a PostgreSQL connection
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    print("Connection established.")  # Log successful connection (you can modify this)

    # Configure logging to write to a log file
    logging.basicConfig(filename='log.txt', level=logging.INFO)
    logging.info("Connection established with Database.")  # Log the same message to the file


    # Additional code for database operations goes here...

except Exception as err:
    # Handle exceptions
    print("Unable to connect to database.")  # Log error (you can modify this)
    print(err)  # Print the specific error message

    # Optionally, write the error message to a file (error.txt)
    with open('error.txt', 'a') as error_file:
        error_file.write(str(err) + '\n')
        
        

Connection established.


In [133]:
#SCHEMA CREATION
import psycopg2
def create_tables():
    commands = (
        # Table for primary ledger
        """
        CREATE TABLE block (
            block_id  UUID DEFAULT gen_random_uuid() PRIMARY KEY NOT NULL,
            Txs_Hash VARCHAR NOT NULL,
            Block VARCHAR NOT NULL,
            Method_S VARCHAR NOT NULL,
            Txn_Fee VARCHAR NOT NULL
        )
        """,
        # Transaction header table for type "exec"
        """
        CREATE TABLE transaction_header_exec (
            transaction_exec_id  UUID DEFAULT gen_random_uuid() PRIMARY KEY NOT NULL ,
            block_id  UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        """
        CREATE TABLE cosmos_MsgExec (
            Msg_Exec_id  UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_exec_id UUID NOT NULL,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_exec_id) REFERENCES transaction_header_exec(transaction_exec_id)
        )
        """,
        """
        CREATE TABLE transaction_header_update (
            transaction_update_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,    
        """
        CREATE TABLE ibc_MsgUpdateClient (
            Msg_Update_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_update_id UUID NOT NULL,
            block_id UUID NOT NULL,
            client_id VARCHAR NOT NULL,
            signer VARCHAR NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_update_id) REFERENCES transaction_header_update(transaction_update_id)
            
        )
        """,
        """
        CREATE TABLE ibc_MsgPacket (
            Msg_Packet_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_update_id UUID NOT NULL,
            block_id UUID NOT NULL,
            Signer VARCHAR NOT NULL,
            Packet_sequence INTEGER NOT NULL,
            Packet_source_port VARCHAR NOT NULL,
            transfer VARCHAR NOT NULL,
            Packet_source_channel VARCHAR NOT NULL,
            Packet_destination_port VARCHAR NOT NULL,
            Packet_destination_channel VARCHAR NOT NULL,
            Packet_Data VARCHAR NOT NULL,
            Proof_commitment VARCHAR NOT NULL,
            Proof_revision_number INTEGER NOT NULL,
            Proof_height INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_update_id) REFERENCES transaction_header_update(transaction_update_id)
        )
        """,
        """
        CREATE TABLE ibc_MsgAcknowledgement (
            Msg_Acknowledgement_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_update_id UUID NOT NULL,
            block_id UUID NOT NULL,
            packet_sequence VARCHAR NOT NULL,
            packet_source_channel VARCHAR NOT NULL,
            packet_destination_port VARCHAR NOT NULL,
            packet_destination_channel VARCHAR NOT NULL,
            packet_data VARCHAR NOT NULL,
            packet_timeout DATE NOT NULL,
            proof_number_height INTEGER NOT NULL,
            proof_height VARCHAR NOT NULL,
            raw_data JSONB,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_update_id) REFERENCES transaction_header_update(transaction_update_id)
            )
        """,
        """
        CREATE TABLE transaction_header_transfer (
            transaction_transfer_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
        )
        """,
        """
        CREATE TABLE ibc_MsgTransfers (
            Msg_Transfer_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_transfer_id UUID NOT NULL,
            block_id UUID NOT NULL,
            sender VARCHAR NOT NULL,
            receiver VARCHAR NOT NULL,
            source_port VARCHAR NOT NULL.
            source_channel VARCHAR NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_transfer_id) REFERENCES transaction_header_transfer(transaction_transfer_id)
        )
        """,
        """
        CREATE TABLE transaction_header_Execute (
            transaction_execute_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #transaction detail table for msg "ExecuteContract"
        """
        CREATE TABLE cosmwasm_MsgExecuteContract (
            Msg_Execute_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_execute_id UUID NOT NULL,
            block_id UUID NOT NULL,
            sender VARCHAR NOT NULL,
            contract VARCHAR NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_execute_id) REFERENCES transaction_header_execute(transaction_execute_id)
            
            
        )
        """,
        # transaction header table for type "Delegate"
        """
        CREATE TABLE transaction_header_delegate (
            transaction_delegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR,
            grantee VARCHAR,
            validator_address VARCHAR,
            denom VARCHAR,
            amount INTEGER,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        #transaction detail table for msg "cosmos_MsgDelegate"
         """
        CREATE TABLE cosmos_MsgDelegate(
            Msg_c_Delegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_delegate_id UUID NOT NULL,
            block_id UUID NOT NULL,
            amount INTEGER NOT NULL,
            denom VARCHAR NOT NULL,
            delegator_address VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_execute_id) REFERENCES transaction_header_execute(transaction_execute_id)          
            
        )
        """,
        # transaction detail table for "alliance_MsgDelegate"
        """
        CREATE TABLE alliance_MsgDelegate(
            Msg_a_Delegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_delegate_id UUID NOT NULL,
            block_id UUID NOT NULL,
            amount INTEGER NOT NULL,
            denom VARCHAR NOT NULL,
            delegator_address VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL, 
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_execute_id) REFERENCES transaction_header_execute(transaction_execute_id)          
            
        )
        """,
        #Transaction header table type "withdraw"
        """
        CREATE TABLE transaction_header_withdraw (
            transaction_withdraw_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        #Transaction detail table msg "WithdrawDelegatorReward"
        """
        CREATE TABLE cosmos_MsgWithdrawDelegatorReward(
            Msg_Delegator_Reward_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_withdraw_id UUID NOT NULL,
            block_id UUID NOT NULL,
            Delegator_Address VARCHAR NOT NULL,
            Validator_Address VARCHAR NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_withdraw_id) REFERENCES transaction_header_withdraw(transaction_withdraw_id)
            
        )
        """,
        """
        CREATE TABLE transaction_header_validator_commission (
            transaction_commission_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        """
        CREATE TABLE cosmos_MsgWithdrawValidatorCommission(
            Msg_Validator_Commission_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_commission_id UUID NOT NULL,
            block_id UUID NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_commission_id) REFERENCES transaction_header_Validator_Commission(transaction_commission_id)
            
        )
        """,
        # transaction header table for type "UnDelegate"
        """
        CREATE TABLE transaction_header_UnDelegate (
            transaction_undelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)

        )
        """,
        #Transaction detail table for msg "cosmos_MsgUndelegate"
        """
        CREATE TABLE cosmos_MsgUndelegate(
            Msg_c_undelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_undelegate_id UUID NOT NULL,
            block_id UUID NOT NULL,
            Delegator_Address VARCHAR NOT NULL,
            Validator_Address VARCHAR NOT NULL,
            Amount VARCHAR NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_undelegate_id) REFERENCES transaction_header_UnDelegate(transaction_undelegate_id)
            
            
        )
        """,
        #Transaction detail table for msg "alliance_MsgUndelegate"
        """
        CREATE TABLE alliance_MsgUndelegate(
            Msg_a_undelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_undelegate_id UUID NOT NULL,
            block_id UUID NOT NULL,
            Delegator_Address VARCHAR NOT NULL,
            Validator_Address VARCHAR NOT NULL,
            Amount VARCHAR NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_undelegate_id) REFERENCES transaction_header_UnDelegate(transaction_undelegate_id)
        )
        """,
        #Transaction header table type "Send"
        """
        CREATE TABLE transaction_header_Send (
            transaction_send_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE cosmos_MsgSend(
            Msg_send_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_send_id UUID NOT NULL,
            block_id UUID NOT NULL,
            raw_data JSONB INT NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_send_id) REFERENCES transaction_header_Send(transaction_send_id)
            
            
        )
        """,
        #Transaction header table type "ClaimDelegate"
        """
        CREATE TABLE transaction_header_ClaimDelegate (
            transaction_claim_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL, 
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE alliance_MsgClaimDelegationRewards(
            Msg_claim_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_claim_id UUID NOT NULL,
            block_id UUID NOT NULL, 
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_claim_id) REFERENCES transaction_header_ClaimDelegate(transaction_claim_id)
        )
        """,
        #Transaction header table type "Grant"
        """
        CREATE TABLE transaction_header_Grant(
            transaction_grant_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE cosmos_MsgGrant(
            Msg_grant_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_grant_id UUID NOT NULL,
            block_id UUID NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_grant_id) REFERENCES transaction_header_Grant(transaction_grant_id)
            
        )
        """,
        #Transaction header table type "BeginDelegate"
        """
        CREATE TABLE transaction_header_BeginDelegate(
            transaction_begin_delegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE cosmos_MsgBeginRedelegate(
            Msg_begin_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_begin_delegate_id  UUID NOT NULL,
            block_id UUID NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_begin_delegate_id) REFERENCES transaction_header_BeginDelegate(transaction_begin_delegate_id)
            
        )
        """,
        #Transaction header table type "Revoke"
        """
        CREATE TABLE transaction_header_Revoke(
            transaction_revoke_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE cosmos_MsgRevoke(
            Msg_Revoke_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_revoke_id UUID NOT NULL,
            block_id UUID NOT NULL,  
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_revoke_id) REFERENCES transaction_header_Revoke(transaction_revoke_id)
            
        )
        """,
        #Transaction header table type "StoreCode"
        """
        CREATE TABLE transaction_header_StoreCode(
            transaction_storecode_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE cosmwasm_MsgStoreCode(
            Msg_Store_Code_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_storecode_id UUID NOT NULL,
            block_id UUID NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_storecode_id) REFERENCES transaction_header_StoreCode(transaction_storecode_id)
            
        )
        """,
        #Transaction header table type "InstantiateContract"
        """
        CREATE TABLE transaction_header_InstantiateContract(
            transaction_instantiate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE cosmwasm_MsgInstantiateContract(
            Msg_instantiate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_instantiate_id UUID NOT NULL,
            block_id UUID NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_instantiate_id) REFERENCES transaction_header_InstantiateContract(transaction_instantiate_id)
        )
        """,
        #Transaction header table for type "Redelegate"
        """
        CREATE TABLE transaction_header_Redelegate(
            transaction_redelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id UUID NOT NULL,
            type VARCHAR INT NOT NULL,
            grantee VARCHAR INT NOT NULL,
            validator_address VARCHAR INT NOT NULL,
            denom VARCHAR INT NOT NULL,
            amount INTEGER INT NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transactio
        """
        CREATE TABLE alliance_MsgRedelegate(
            Msg_Redegegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_redelegate_id UUID NOT NULL,
            block_id UUID NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id),
            FOREIGN KEY(transaction_redelegate_id) REFERENCES transaction_header_Redelegate(transaction_redelegate_id)
            
        )
        """
                # Transaction detail table for msg "exec"
 
              
    )

    try:
        # Replace with your actual database connection parameters
        conn = psycopg2.connect(
            dbname='blockchain',
            user='postgres',
            password='auth2020',
            host='localhost',
            port='5432'
        )

        with conn:
            with conn.cursor() as cur:
                for command in commands:
                    cur.execute(command)

    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

if __name__ == '__main__':
    create_tables()     
        

syntax error at or near ")"
LINE 10:         )
                 ^


In [134]:
       
        # Transaction header table for type "Update"
        
        # Transaction detail table for msg "Update"
        
        # Additional table: ibc_MsgPacket
        
         # transaction detail table for msg "Acknowledgement"
        
        
        
        # transaction header table for type "Execute"
        